In [1]:
# Clear Attempts Data - Jupyter Notebook
# Save this as clear_attempts.ipynb

import firebase_admin
from firebase_admin import credentials, firestore
from pathlib import Path
import json

# Configuration - match your Streamlit app
SERVICE_ACCOUNT_PATH = "serviceAccountKey.json"

# Initialize Firebase
if not firebase_admin._apps:
    cred = credentials.Certificate(SERVICE_ACCOUNT_PATH)
    firebase_admin.initialize_app(cred)

db = firestore.client()

def clear_all_attempts():
    """Clear all attempt documents from Firestore"""
    print("🔄 Starting to clear attempts...")
    
    try:
        # Get all documents in the 'attempts' collection
        attempts_ref = db.collection('attempts')
        attempts = attempts_ref.stream()
        
        count = 0
        for attempt in attempts:
            attempt.reference.delete()
            count += 1
            if count % 10 == 0:
                print(f"   Deleted {count} attempts...")
        
        print(f"✅ Successfully deleted {count} attempt records")
        return count
        
    except Exception as e:
        print(f"❌ Error clearing attempts: {e}")
        return 0

def clear_user_attempt_history():
    """Clear attempt history from user documents but keep user accounts"""
    print("🔄 Clearing user attempt histories...")
    
    try:
        # Get all users
        users_ref = db.collection('users')
        users = users_ref.stream()
        
        count = 0
        for user in users:
            user_data = user.to_dict()
            
            # Reset attempt-related fields but keep account info
            updates = {}
            
            if 'best_score' in user_data:
                updates['best_score'] = 0
            if 'answered_questions' in user_data:
                updates['answered_questions'] = []
            if 'total_attempts' in user_data:
                updates['total_attempts'] = 0
            if 'last_attempt_date' in user_data:
                updates['last_attempt_date'] = None
                
            if updates:
                user.reference.update(updates)
                count += 1
                print(f"   Reset user: {user_data.get('display_name', user.id)}")
        
        print(f"✅ Successfully reset {count} user records")
        return count
        
    except Exception as e:
        print(f"❌ Error clearing user histories: {e}")
        return 0

def get_database_stats():
    """Show current database statistics"""
    print("📊 Current Database Stats:")
    print("-" * 30)
    
    try:
        # Count attempts
        attempts_ref = db.collection('attempts')
        attempts = list(attempts_ref.stream())
        print(f"Total attempts: {len(attempts)}")
        
        # Count users  
        users_ref = db.collection('users')
        users = list(users_ref.stream())
        print(f"Total users: {len(users)}")
        
        # Show user scores
        if users:
            print("\nUser Scores:")
            for user in users:
                user_data = user.to_dict()
                name = user_data.get('display_name', user.id)
                score = user_data.get('best_score', 0)
                answered = len(user_data.get('answered_questions', []))
                print(f"  {name}: {score} points, {answered} questions answered")
        
    except Exception as e:
        print(f"❌ Error getting stats: {e}")

# Main execution
if __name__ == "__main__":
    print("🗑️  FIREBASE DATA CLEANER")
    print("=" * 40)
    
    # Show current stats
    get_database_stats()
    print()
    
    # Ask for confirmation
    choice = input("What would you like to clear?\n"
                  "1. All attempts only (keep user scores)\n"
                  "2. All attempts + reset user scores\n"
                  "3. Just show stats\n"
                  "Choice (1/2/3): ").strip()
    
    if choice == "1":
        print("\n🔄 Clearing attempts only...")
        cleared = clear_all_attempts()
        print(f"\n✅ Done! Cleared {cleared} attempts")
        
    elif choice == "2":
        print("\n🔄 Clearing everything...")
        attempts_cleared = clear_all_attempts()
        users_reset = clear_user_attempt_history()
        print(f"\n✅ Done! Cleared {attempts_cleared} attempts and reset {users_reset} users")
        
    elif choice == "3":
        print("\n📊 Stats only - no changes made")
        
    else:
        print("❌ Invalid choice - no changes made")
    
    print("\n" + "=" * 40)
    print("📊 FINAL DATABASE STATS:")
    get_database_stats()

# Quick functions you can run individually:

def quick_clear_all():
    """Quick function to clear everything - no prompts"""
    attempts = clear_all_attempts()
    users = clear_user_attempt_history()
    print(f"🗑️ Cleared {attempts} attempts, reset {users} users")

def quick_clear_attempts_only():
    """Quick function to clear attempts only - no prompts"""
    cleared = clear_all_attempts()
    print(f"🗑️ Cleared {attempts} attempts")

def quick_stats():
    """Quick function to show stats"""
    get_database_stats()

# Uncomment one of these lines to run directly:
# quick_clear_all()
# quick_clear_attempts_only() 
quick_stats()

🗑️  FIREBASE DATA CLEANER
📊 Current Database Stats:
------------------------------
Total attempts: 9
Total users: 20

User Scores:
  a b: 12 points, 12 questions answered
  Soham: 10 points, 8 questions answered
  strat1: 24 points, 24 questions answered
  Dinesh: 0 points, 0 questions answered
  figure it out: 14 points, 8 questions answered
  Raghav: 3 points, 3 questions answered
  henry: 3 points, 3 questions answered
  Umang: 5 points, 5 questions answered
  Harish: 57 points, 61 questions answered
  henry: 19 points, 17 questions answered
  Dark Clouds: 5 points, 10 questions answered
  Manu: 24 points, 43 questions answered
  mohit: 15 points, 32 questions answered
  Karmuhilan K J: 36 points, 80 questions answered
  NS: 9 points, 9 questions answered
  Prachi: 7 points, 6 questions answered
  rachan: 16 points, 29 questions answered
  test: 2 points, 16 questions answered
  Kshitij Kumar: 17 points, 14 questions answered
  TS: 20 points, 102 questions answered


🔄 Clearing ever